In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define player data
players = []
for i in range(0, len(data), 3):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+1].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
    }
    players.append(player)

# Process inputs and targets for each player
X_train, Y_train, X_test, Y_test = [], [], [], []
 
for player in players[:8]:
    player_inputs = torch.from_numpy(player['defense_ranks']).view(1, 15, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score']).view(1, 15, 1).float()), dim=2)
    X_train.append(player_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores']).view(1, 15, 1).float())
  
for player in players[8:9]:
    player_test_inputs = torch.from_numpy(player['defense_ranks']).view(1, 15, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score']).view(1, 15, 1).float()), dim=2)
    X_test.append(player_test_inputs)  
    Y_test.append(torch.from_numpy(player['fantasy_scores']).view(1, 15, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0)
Y_train = torch.cat(Y_train, dim=0)
X_test = torch.cat(X_test, dim=0)
Y_test = torch.cat(Y_test, dim=0)

In [11]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 50
learning_rate = 0.0001

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
total = 0
params = list(model.parameters())
for i in params:
    total += len(i)
total

302

In [15]:
# Training loop for each player with batch size of 2
batch_size = 1
num_players = len(X_train)
num_batches = num_players // batch_size # Only full batches
epoch_runs = 20
batch_runs = 10

# Training loop
for epoch_run in range(epoch_runs):
    for batch in range(num_batches):
        batch_inputs = X_train[batch * batch_size: (batch + 1) * batch_size]
        batch_targets = Y_train[batch * batch_size: (batch + 1) * batch_size]

        # number of times a batch runs before running the next batch
        for batch_run in range(batch_runs):
            # Forward pass
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print the loss for every epoch
        if epoch_run + 1 == epoch_runs:
            print(f"Batch {batch+1}/{num_batches}, Epoch {epoch_run+1}/{epoch_runs}, Loss: {loss.item()}")
    
# Make predictions for the player's test inputs
with torch.no_grad():
    player_predicted = model(X_test)

print() # blank line
# Print the predictions and original scores for the player
for i in range(len(X_test[0])):
    for j in range(len(X_test)):
        print(f"Player {j+1}: Skill Score: {round(X_test[j][i][1].item(), 2)}, Defense Rank: {X_test[j][i][0]}, " + 
              f"Predicted Fantasy Score: {round(player_predicted[j][i].item(), 2)}, Actual Fantasy Score: {round(Y_test[j][i].item(), 2)}")
    print() # blank line

Batch 1/8, Epoch 20/20, Loss: 111.51897430419922
Batch 2/8, Epoch 20/20, Loss: 52.35102844238281
Batch 3/8, Epoch 20/20, Loss: 54.28216552734375
Batch 4/8, Epoch 20/20, Loss: 49.0099983215332
Batch 5/8, Epoch 20/20, Loss: 40.85084533691406
Batch 6/8, Epoch 20/20, Loss: 37.50271987915039
Batch 7/8, Epoch 20/20, Loss: 48.64801025390625
Batch 8/8, Epoch 20/20, Loss: 73.85680389404297

Player 1: Skill Score: 14.64, Defense Rank: 12.0, Predicted Fantasy Score: 22.0, Actual Fantasy Score: 10.8

Player 1: Skill Score: 14.64, Defense Rank: 5.0, Predicted Fantasy Score: 14.43, Actual Fantasy Score: 3.6

Player 1: Skill Score: 14.64, Defense Rank: 29.0, Predicted Fantasy Score: 25.1, Actual Fantasy Score: 13.6

Player 1: Skill Score: 14.64, Defense Rank: 16.0, Predicted Fantasy Score: 22.53, Actual Fantasy Score: 8.6

Player 1: Skill Score: 14.64, Defense Rank: 32.0, Predicted Fantasy Score: 25.77, Actual Fantasy Score: 24.1

Player 1: Skill Score: 14.64, Defense Rank: 20.0, Predicted Fantasy Sc